In [12]:
LDA SIDT3		;THRUST IN SPACE
JSR SNDON
JSR MODSND

*=$2000
; -----------------------------------------------
; Enable a sound
; -----------------------------------------------
SNDON:
    TAY            ;USE AS INDEX
    LDX NCHANL-1            ;NO.
    
SNDBEGIN                ;--;LOOP FOR EACH 1/2 CHANNEL
    LDA PNTRS,Y
    BEQ NEXT
    
    STX SINDEX
    STA POINT,X        ;IF NOT SET UP POINTER 
    LDA 1
    STA FRAMES,X        ;DUMMY START, NO SOUND
    STA COUNT,X        ;TILL MODSND STORES TO POKEY
    LDA $FF
    STA SINDEX
    
NEXT:
    DEY
    DEX
    BNE SNDBEGIN
NWSNON:
    RTS

; -----------------------------------------------
; Update the sound
; -----------------------------------------------
MODSND:
    LDX NCHANL-1            ;8 CHANNELS
    
BEGIN:
    LDA POINT,X
    BEQ DONEXT1         ; IGNORE ZERO POINT
    CPX SINDEX
    BEQ DONEXT1         ; IGNORE IF NO SOUND
    DEC FRAMES,X
    BNE DONEXT1         ; IGNORE IF FRAMES NOT FINISHED YET
    DEC COUNT,X
    BNE OLDGROUP       ;MOVE TO NEXT CHANGE GROUP?
    JMP RESOUN
    
DONEXT1
    JMP DONEXT
    
RESOUN:    
    INC POINT,X        ;YES. START VALUE
    INC POINT,X
    LDA POINT,X
    ASL
    TAY
    BCS SIGNED
    
    LDA SOUND+STVAL,Y
    STA CURRENT,X
    LDA SOUND+NUMBER,Y
    STA COUNT,X
    LDA SOUND+FRCNT,Y
    JMP CHKCHNL
    
SIGNED:
    LDA SOUND+STVAL+100,Y
    STA CURRENT,X
    LDA SOUND+NUMBER+100,Y
    STA COUNT,X
    LDA SOUND+FRCNT+100,Y
    
CHKCHNL:
    STA FRAMES,X
    BNE UPDCHNL            ;CHANNEL STILL ALIVE?
    
    STA POINT,X        ;NO. KILL IT
    
    LDA CURRENT,X
    BEQ UPDCHNL            ;RESTART CHANNEL?
    
    STA POINT,X        ;YES. UPDATE PTR. WITH RESTART LOC
    BNE RESOUN
    
    JMP UPDCHNL
                ;NO. MAKE CHANGE IN OLD GROUP
OLDGROUP:
    ASL
    TAY
    BCS OLDSIGNED
    LDA SOUND+FRCNT,Y
    STA FRAMES,X
    LDA SOUND+CHANGE,Y
    JMP OCHKCHNL
    
OLDSIGNED:
    LDA SOUND+FRCNT+100,Y
    STA FRAMES,X
    LDA SOUND+CHANGE+100,Y
    
OCHKCHNL:
    LDY CURRENT,X
    CLC
    ADC CURRENT,X
    STA CURRENT,X
    TXA
    LSR
    BCC UPDCHNL

    TYA
    EOR CURRENT,X
    AND $F0
    EOR CURRENT,X
    STA CURRENT,X
    
UPDCHNL:
    LDA CURRENT,X        ;UPDATE POKEY AUDIO CHANNEL
    CPX 8
    BCC UPCH
    
    STA AUDCTL,X
    JMP DONEXT
    
UPCH:
    STA AUDF1,X

DONEXT:
    DEX
    BEQ ENDLOOP
    JMP BEGIN
ENDLOOP:
    RTS


NCHANL = #$10
POKEY = $00C0
AUDF1    =POKEY
AUDC1    =POKEY+1
AUDCTL    =POKEY+8
ALLPOT    =POKEY+8
RANDOM    =POKEY+$0A
POTGO    =POKEY+$0B
SKCTL    =POKEY+$0F

STVAL    =0            ;VALUE TO START SEQUENCE
FRCNT    =1            ;# OF FRAMES BEFORE ANY CHANGE
CHANGE    =2            ;AMOUNT OF CHANGE
NUMBER    =3            ;TOTAL NUMBER OF CHANGES IN THIS SEQUENCE

SOUND:
T51F:
    .BYTE $C0,8,4,$10
    .BYTE 0,0
T51A:
    .BYTE $A6,$20,$F8,4
    .BYTE 0,0
PNTRS:
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$00,$00
.BYTE $35,$38,$00,$00
.BYTE $00,$00,$00,$00
SIDLO
.BYTE $00,$00,$47,$4A
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$00,$00
SIDEX
.BYTE $00,$00,$00,$00
.BYTE $0D,$10,$00,$00
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$00,$00
SIDLA
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$65,$68
.BYTE $00,$00,$00,$00
SIDPU
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$21,$32
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$00,$00
SIDWP
.BYTE $13,$1A,$00,$00
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$00,$00
SIDDI
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$53,$56
.BYTE $00,$00,$00,$00
SIDT2
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$59,$5C
.BYTE $00,$00,$00,$00
SIDT3
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$3B,$3E
SIDES
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$00,$00
.BYTE $41,$44,$00,$00
SIDEL
.BYTE $4D,$50,$00,$00
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$00,$00
SIDSL
.BYTE $5F,$62,$00,$00
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$00,$00
SIDS3
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$00,$00
.BYTE $00,$00,$6D,$6D
.BYTE $00,$00,$00,$00
SIDPO

*=$6000
POINT:     .BYTE $00 x 16
CURRENT:   .BYTE $00 x 16
FRAMES:    .BYTE $00 x 16
COUNT:     .BYTE $00 x 16
SINDEX     .BYTE $00


CPU Register Legend:                                   
C = Carry,    Z = Zero, I = Interrupt, D = Decimal Mode
V = Overflow, N = Negative                             
                                                       
Execution History (Values are after execution.)        
                                                       
                       | CPU Register | Program           
Instruction |  A  X  Y | C Z I D V N  | Counter           
----------- | -- -- -- | -----------  | -------           
LDA $2153   | 00 00 00 | 0 1 0 0 0 0  | 049155 
JSR $2000   | 00 00 00 | 0 1 0 0 0 0  | 008192 
TAY         | 00 00 00 | 0 1 0 0 0 0  | 008193 
LDX #000F   | 00 0F 00 | 0 0 0 0 0 0  | 008195 
LDA $20D3   | 00 0F 00 | 0 1 0 0 0 0  | 008198 
BEQ $0013   | 00 0F 00 | 0 1 0 0 0 0  | 008219 
DEY         | 00 0F FF | 0 0 0 0 0 1  | 008220 
DEX         | 00 0E FF | 0 0 0 0 0 0  | 008221 
BNE $00E4   | 00 0E FF | 0 0 0 0 0 0  | 008195 
LDA $20D3   | 00 0E FF | 0 1 0 0 0 0  | 008198 
BEQ $00